In [1]:
#This will be a modified version of Jaime's Brain Cleaning

In [2]:
import pandas as pd
import re
import numpy as np
import requests, sys
import json
import time
import pickle 
import os
import glob
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from statannot import add_stat_annotation
import math
from matplotlib.patches import Patch
from dna_features_viewer import GraphicFeature, GraphicRecord
from matplotlib_venn import venn2,venn2_circles

In [4]:
brain_df = pd.read_csv('jaime_db/strictly.csv')
    
csf_df = pd.read_csv('jaime_db/csfdf.csv')
csf_no_chen = pd.read_csv('jaime_db/csf_no_chen.csv')

overlapsdf = pd.read_csv('jaime_db/overlapsdf.csv')
theoriticaldf = pd.read_csv('jaime_db/theoreticalscript.csv')
theoriticaldf_high = pd.read_csv('jaime_db/theoreticalscript_high.csv')
theoriticaldf_one = pd.read_csv('jaime_db/theoreticalscript_one.csv')
theoriticaldf_two = pd.read_csv('jaime_db/theoreticalscript_two.csv')

# opens dict with uniprot information
with open('Uniprotdata', 'rb') as handle:
    proteindic = pickle.load(handle)

In [5]:
#Below is EDA on the brain_df

In [6]:
len(np.unique(brain_df.Protein))

6379

In [7]:
np.unique(brain_df.CSF)

array([False])

In [8]:
overlapsdf.head()

,Protein,TP,Start,End,Classification
0,A0AVT1,FDLNEPLHLSFLQNAAK,746,762,Negative
1,A0FGR8,ISSNPNPVVQMSVGHK,553,568,Negative
2,A0MZ66,EQAIGEYEDLR,17,27,Negative
3,A0MZ66,ISMLYMAK,99,106,Negative
4,A0MZ66,DQIVSVQEEK,148,157,Negative


In [9]:
# finds sequence of protein in proteindic acquired from uniprot
def findseq(row,i):
    seq = None
    if row['Protein'] in proteindic:
        if proteindic[row['Protein']] != None:
            seq = proteindic[row['Protein']][i]
    return seq

# returns rows where the peptide is only found once in the protein
def findallseq(x):
    if len(re.findall(x['Peptide_Sequence'],x['Protein_Sequence'])) == 1:
        return x

In [10]:
brain_df['Protein_Sequence'] = brain_df.apply(lambda x: findseq(x,0),axis =1)
print('Before removing proteins not valid in uniprot',len(brain_df),'sequences and',len(brain_df['Protein'].unique()),'proteins found in df')
brain_df = brain_df[brain_df['Protein_Sequence'].notna()]
print('Afterwards',len(brain_df),'sequences and',len(brain_df['Protein'].unique()),'proteins remained')
pl = brain_df.apply(lambda x: len(x['Protein_Sequence']),axis =1)
brain_df = brain_df.assign(Protein_Length  = pl)

Before removing proteins not valid in uniprot 78050 sequences and 6379 proteins found in df
Afterwards 77993 sequences and 6369 proteins remained


In [11]:
#remove sequences which are found twice in the same protein
brain_df = brain_df.apply(lambda x: findallseq(x),axis=1)
print('Removing peptides which are found more than once in the same protein removes', brain_df['Peptide_Sequence'].isna().sum(), 'peptides, this can be peptides found twice in the same protein or peptides not found, due to being from an isoform, where the canonical protein sequence is used')
brain_df.dropna(inplace=True)
print('leaving a total of',len(brain_df),'peptides left')
pp = brain_df.apply(lambda x: x['Protein_Sequence'].find(x['Peptide_Sequence']),axis =1)
brain_df = brain_df.assign(Position_in_Protein = pp)

print(len(brain_df[brain_df['Position_in_Protein']==-1]),'peptides have no position in related protein, most likely due to being an isoform peptide and being related to the canonical protein, -> these are already removed by the step above')
brain_df = brain_df[brain_df['Position_in_Protein']!=-1]
print('Afterwards',len(brain_df),'sequences and',len(brain_df['Protein'].unique()),'proteins remained')

Removing peptides which are found more than once in the same protein removes 135 peptides, this can be peptides found twice in the same protein or peptides not found, due to being from an isoform, where the canonical protein sequence is used
leaving a total of 77833 peptides left
0 peptides have no position in related protein, most likely due to being an isoform peptide and being related to the canonical protein, -> these are already removed by the step above
Afterwards 77833 sequences and 6364 proteins remained


In [12]:
brain_df_filtered = brain_df.groupby('Protein').filter(lambda g: len(g) >= 2)

In [13]:
len(np.unique(brain_df_filtered.Protein))

5497

In [14]:
brain_df_filtered

,Peptide_Sequence,Sequence_Length,Protein,Brain,CSF,Author,Detectability,Detection_Probability,Protein_Sequence,Protein_Length,Position_in_Protein
0,AAAAAAAAAAAAAAAGAGAGAK,22.0,P55011,True,False,"Faigle, Rydbirk, Duong",0.0,0.122071,MEPRPTAPSSGAPGLAGVGETPSAAALAAARVELPGTAVPSVPEDA...,1212.0,92
1,AAAAAAAAAAAPPAPPEGASPGDSAR,26.0,Q8WXD9,True,False,"Faigle, Rydbirk, Duong",1.0,0.581326,MGKEQELVQAVKAEDVGTAQRLLQRPRPGKAKLLGSTKKINVNFQD...,1431.0,1344
2,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29.0,P37108,True,False,"Rydbirk, Duong",1.0,0.807786,MVLLESEQFLTELTRLFQKCRTSGSVYITLKKYDGRTKPIPKKGTV...,136.0,107
3,AAAAAAAAVPSAGPAGPAPTSAAGR,25.0,Q9Y4H2,True,False,"Faigle, Duong",1.0,0.501463,MASPPRHGPPGPASGDGPNLNNNNNNNNHSVRKCGYLRKQKHGHKR...,1338.0,693
4,AAAAAAALQAK,11.0,P36578,True,False,"Faigle, Rydbirk, Duong",1.0,0.673549,MACARPLISVYSEKGESSGKNVTLPAVFKAPIRPDIVNFVHTNLRK...,427.0,353
...,...,...,...,...,...,...,...,...,...,...,...
78020,YYQTIGNHASYYK,13.0,Q9UNM6,True,False,"Faigle, Rydbirk",1.0,0.684231,MKDVPGFLQQSQNSGPGQPAVWHRLEELYTKKLWHQLTLQVLDFVQ...,376.0,161
78021,YYRPTEVDFLQGDCTK,16.0,O60547,True,False,"Rydbirk, Duong",1.0,0.785365,MAHAPARCPSARGSGDGEMGKPRNVALITGITGQDGSYLAEFLLEK...,372.0,322
78022,YYSDIGK,7.0,Q9HCM2,True,False,"Faigle, Rydbirk, Duong",1.0,0.780594,MKAMPWNWTCLLSHLLMVGMGSSTLLTRQPAPLSQKQRSFVTFRGE...,1894.0,1813
78023,YYSDLFSYCDIESTK,15.0,Q96D71,True,False,"Rydbirk, Duong",1.0,0.740213,MEGLTLSDAEQKYYSDLFSYCDIESTKKVVVNGRVLELFRAAQLPN...,796.0,12


In [15]:
#This was a bad idea I had to filter the database via z-score. 
#This wont work since the data is not normally distributed.

#from scipy import stats
#brain_df_filtered = brain_df_filtered[np.abs(stats.zscore(brain_df_filtered.Protein_Length) < 3)]

In [16]:
print(brain_df_filtered['Protein_Length'].quantile([0.25, 0.75]))

0.25     394.0
0.75    1084.0
Name: Protein_Length, dtype: float64


In [17]:
IQR = 1084-394
lower_limit = 394 - (1.5*IQR)
upper_limit = 1084 + (1.5*IQR)

print(lower_limit)
print(upper_limit)

-641.0
2119.0


In [18]:
condition = brain_df_filtered['Protein_Length']<2119

In [19]:
brain_df_filtered = brain_df_filtered[condition]

In [20]:
#this is just a sanity check to ensure that the previous outliers are removed.
brain_df_filtered.max(axis=0)['Protein_Length']

2115.0

In [22]:
len(np.unique(brain_df_filtered.Protein))

5370

In [20]:
#This is now dealing with the uniprot data
d = []
proteinsummary = []
for key in proteindic.keys():
    # skips proteins not valid in uniprot or with no feature information
#     print(proteindic.get(key)[3])
    #print(key)
    if proteindic[key] == None or proteindic[key][3].get('countByCommentType') == None:
         proteinsummary.append(
            {
                'Protein': key,
                'Function': None,
                'Subunit': None,
                'Interaction':  None,
                'Subcellular_Location':  None,
                'Isoforms': None,
                'PTM': None,
                'Similarity':  None,
                'Miscellaneous':  None,
                
                'Function': None,
                'Subunit': None,
                'Interaction':  None,
                'Subcellular_Location':  None
            }
        )
    else:
        proteinsummary.append({
            'Protein': key,
            'Function': proteindic.get(key)[3].get('countByCommentType').get('FUNCTION'),
            'Subunit': proteindic.get(key)[3].get('countByCommentType').get('SUBUNIT'),
            'Domain': proteindic.get(key)[3].get('countByCommentType').get('DOMAIN'),
            'Interaction': proteindic.get(key)[3].get('countByCommentType').get('INTERACTION'),
            'Subcellular_Location': proteindic.get(key)[3].get('countByCommentType').get('SUBCELLULAR LOCATION'),
            'Isoforms': proteindic.get(key)[3].get('countByCommentType').get('ALTERNATIVE PRODUCTS'),
            'PTM': proteindic.get(key)[3].get('countByCommentType').get('PTM'),
            'Similarity': proteindic.get(key)[3].get('countByCommentType').get('SIMILARITY'),
            'Miscellaneous': proteindic.get(key)[3].get('countByCommentType').get('MISCELLANEOUS'),
            'Tissue_Specificity': proteindic.get(key)[3].get('countByCommentType').get('TISSUE SPECIFICITY'),
            'Disease': proteindic.get(key)[3].get('countByCommentType').get('DISEASE'),
            'Caution': proteindic.get(key)[3].get('countByCommentType').get('CAUTION'),
            'Sequence_Caution': proteindic.get(key)[3].get('countByCommentType').get('SEQUENCE CAUTION'),
            'Pathway': proteindic.get(key)[3].get('countByCommentType').get('PATHWAY'),
            'Catalytic_Activity': proteindic.get(key)[3].get('countByCommentType').get('CATALYTIC ACTIVITY'),
            'Activity_Regulation': proteindic.get(key)[3].get('countByCommentType').get('ACTIVITY REGULATION'),
            'Biophysicohemical_Properties': proteindic.get(key)[3].get('countByCommentType').get('BIOPHYSICOCHEMICAL PROPERTIES'),
            'Interaction': proteindic.get(key)[3].get('countByCommentType').get('INTERACTION'),
            'Induction': proteindic.get(key)[3].get('countByCommentType').get('INDUCTION'),
            'Mass_Spec': proteindic.get(key)[3].get('countByCommentType').get('MASS SPECTROMETRY'),
            'Developmental_Stage': proteindic.get(key)[3].get('countByCommentType').get('DEVELOPMENTAL STAGE'),
            'Cofactor': proteindic.get(key)[3].get('countByCommentType').get('COFACTOR'),
            'Polymorphism': proteindic.get(key)[3].get('countByCommentType').get('POLYMORPHISM'),
            'Web_Resource': proteindic.get(key)[3].get('countByCommentType').get('WEB RESOURCE')
            
            
        })
        
    if proteindic[key] == None or proteindic[key][2] == None:
        d.append(
            {
                'Protein': key,
                'Type': None,
                'Description': None,
                'Start':  None,
                'End':  None
            }
        )
    else: 
        for p in proteindic[key][2]:
        # df index each protein -> columns are keys
            d.append(
                {
                    'Protein': key,
                    'Type': p['type'],
                    'Description': p.get('description'),
                    'Start':  p['location']['start']['value']-1 if p['location']['start']['value'] != None else p['location']['start']['value'],
                    'End':  p['location']['end']['value']-1 if p['location']['end']['value'] != None else p['location']['end']['value']
                }
            )
proteindomains = pd.DataFrame(d)
proteinsummaries = pd.DataFrame(proteinsummary)
print(proteindomains.isna().sum())

Protein          0
Type           220
Description    220
Start          289
End            297
dtype: int64


In [21]:
proteindomains[proteindomains['Type']=='Domain']

,Protein,Type,Description,Start,End
637,P05067,Domain,E1,27.0,188.0
638,P05067,Domain,BPTI/Kunitz inhibitor,290.0,340.0
639,P05067,Domain,E2,373.0,564.0
988,Q8IZP0,Domain,t-SNARE coiled-coil homology,44.0,106.0
989,Q8IZP0,Domain,SH3,445.0,504.0
...,...,...,...,...,...
414670,Q8N8U9,Domain,TIL,628.0,681.0
414686,E7EW49,Domain,TOG,6.0,231.0
414688,E7EW49,Domain,TOG,317.0,550.0
414689,E7EW49,Domain,TOG,867.0,1104.0


In [22]:
 len(np.unique(proteindomains.Protein))

10799

In [23]:
invalidproteins = proteindomains[proteindomains['Type'].isna()]
proteindomains = proteindomains[proteindomains['Type'].notna()]

print('In total',len(invalidproteins['Protein'].unique()),'proteins were not valid in the uniprot db or had no feature info')

In total 220 proteins were not valid in the uniprot db or had no feature info


In [24]:
#Here's another sanity check-point. I just want to see how much the database I lose by removing these.
print('This is the current amount of brain only peptides:', len(brain_df_filtered))

This is the current amount of brain only peptides: 70192


In [25]:
invalidproteins

,Protein,Type,Description,Start,End
14959,P30042,None,None,NaN,NaN
21371,P08107,None,None,NaN,NaN
45118,A6NL28,None,None,NaN,NaN
47848,Q6ZRH9,None,None,NaN,NaN
67686,P0DN79,None,None,NaN,NaN
...,...,...,...,...,...
414188,A0A0A0MSQ6,None,None,NaN,NaN
414189,A0A087WSY9,None,None,NaN,NaN
414383,A0A096LPI6,None,None,NaN,NaN
414384,A0A087WZH7,None,None,NaN,NaN


In [26]:
#Dropping those proteins without associated uniprot data.
present = brain_df_filtered['Protein'].isin(invalidproteins.Protein)

In [27]:
#Turns out none of those proteins were even in the brain data to begin with. This step is in essence useless.
print(present.unique())

[False]


In [28]:
#remaining NA's are due to residues at start or end of protein
print(proteindomains.isna().sum())
proteindomains[(proteindomains['Start'].isna())&(proteindomains['End'].isna())]
print(len(proteindomains[proteindomains['Description']=='']))
proteindomains['Description'] = proteindomains.apply(lambda p: p['Type'] if p['Description']=='' else p['Description'] ,axis=1)
print(len(proteindomains[proteindomains['Description']=='']))
proteindomains

Protein         0
Type            0
Description     0
Start          69
End            77
dtype: int64
159141
0


,Protein,Type,Description,Start,End
0,P31946,Chain,14-3-3 protein beta/alpha,0.0,245.0
1,P31946,Initiator methionine,Removed; alternate,0.0,0.0
2,P31946,Chain,"14-3-3 protein beta/alpha, N-terminally processed",1.0,245.0
3,P31946,Site,Interaction with phosphoserine on interacting ...,57.0,57.0
4,P31946,Site,Interaction with phosphoserine on interacting ...,128.0,128.0
...,...,...,...,...,...
414697,E7EW49,Region,Disordered,1171.0,1214.0
414698,E7EW49,Compositional bias,Polar residues,681.0,700.0
414699,E7EW49,Compositional bias,Polar residues,721.0,764.0
414700,E7EW49,Compositional bias,Polar residues,843.0,865.0


In [29]:
#Below here shows some categories and properties of proteins. Really useful.
print(proteindomains['Description'].value_counts()[:50])
signalproteins = proteindomains[proteindomains['Description']=='Signal']['Protein']

Beta strand                                                                        57120
Helix                                                                              57048
Phosphoserine                                                                      24264
Binding site                                                                       16808
Disordered                                                                         15505
Turn                                                                               14449
Disulfide bond                                                                      8347
Polar residues                                                                      7628
N-linked (GlcNAc...) asparagine                                                     6968
in isoform 2                                                                        6498
Helical                                                                             6213
Basic and acidic resi

In [30]:
proteinsummaries.drop(proteinsummaries[~proteinsummaries['Protein'].isin(brain_df_filtered['Protein'].tolist())].index, inplace = True)
print(len(proteinsummaries[proteinsummaries['Isoforms']>=1]))
proteinsummaries

3387


,Protein,Function,Subunit,Domain,Interaction,Subcellular_Location,Isoforms,PTM,Similarity,Miscellaneous,...,Pathway,Catalytic_Activity,Activity_Regulation,Biophysicohemical_Properties,Induction,Mass_Spec,Developmental_Stage,Cofactor,Polymorphism,Web_Resource
0,P31946,1.0,3.0,NaN,43.0,2.0,2.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,P62258,1.0,2.0,NaN,36.0,1.0,2.0,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Q04917,1.0,1.0,NaN,32.0,NaN,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P61981,1.0,1.0,NaN,144.0,1.0,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P27348,1.0,1.0,NaN,26.0,1.0,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8532,Q9BUW7,1.0,NaN,NaN,11.0,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8550,P0CG08,1.0,1.0,NaN,1.0,1.0,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8569,P22792,1.0,1.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8622,Q16585,1.0,1.0,NaN,16.0,1.0,2.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [31]:
#This is just adding information to see if the brain peptides are also found in the CSF.
brain_df_filtered['CSF'] = brain_df_filtered['Peptide_Sequence'].isin(csf_df['Peptide_Sequence'])

/var/folders/zx/9wq85h090yj8pmk6sbvl93n80000gn/T/ipykernel_1500/21097524.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brain_df_filtered['CSF'] = brain_df_filtered['Peptide_Sequence'].isin(csf_df['Peptide_Sequence'])


In [32]:
#another sanity check point
brain_df_filtered
print(brain_df_filtered['CSF'].value_counts())

False    60239
True      9953
Name: CSF, dtype: int64


In [33]:
positive = sum(brain_df_filtered['CSF'] == True)
print('Of the',len(brain_df_filtered),' peptides,',positive,'were found in the CSF')
print('This means ',round((positive)/len(brain_df_filtered)*100),'% of the brain peptides also occur in the csf')

Of the 70192  peptides, 9953 were found in the CSF
This means  14 % of the brain peptides also occur in the csf


In [34]:
mixed_df = brain_df_filtered[brain_df_filtered['Brain'] & brain_df_filtered['CSF']]

In [35]:
mixed_df

,Peptide_Sequence,Sequence_Length,Protein,Brain,CSF,Author,Detectability,Detection_Probability,Protein_Sequence,Protein_Length,Position_in_Protein
0,AAAAAAAAAAAAAAAGAGAGAK,22.0,P55011,True,True,"Faigle, Rydbirk, Duong",0.0,0.122071,MEPRPTAPSSGAPGLAGVGETPSAAALAAARVELPGTAVPSVPEDA...,1212.0,92
3,AAAAAAAAVPSAGPAGPAPTSAAGR,25.0,Q9Y4H2,True,True,"Faigle, Duong",1.0,0.501463,MASPPRHGPPGPASGDGPNLNNNNNNNNHSVRKCGYLRKQKHGHKR...,1338.0,693
4,AAAAAAALQAK,11.0,P36578,True,True,"Faigle, Rydbirk, Duong",1.0,0.673549,MACARPLISVYSEKGESSGKNVTLPAVFKAPIRPDIVNFVHTNLRK...,427.0,353
12,AAAASAAEAGIATTGTEDSDDALLK,25.0,P55036,True,True,"Faigle, Rydbirk, Duong",1.0,0.953609,MVLESTMVCVDNSEYMRNGDFLPTRLQAQQDAVNIVCHSKTRSNPE...,377.0,237
27,AAAEQLR,7.0,O94766,True,True,"Rydbirk, Duong",1.0,0.767400,MKLKLKNVFLAYFLVSIAGLLYALVQLGQPCDCLPPLRAAAEQLRQ...,335.0,38
...,...,...,...,...,...,...,...,...,...,...,...
77986,YYIAASYVK,9.0,Q92820,True,True,"Faigle, Rydbirk, Duong",1.0,0.686800,MASPGCLLCVLGLLLCGAASLELSRPHGDTAKKPIIGILMQKCRNK...,318.0,53
77999,YYLQGAK,7.0,Q14624,True,True,"Faigle, Rydbirk",1.0,0.853688,MKPPRPVRTCSKVLVLLSLLAIHQTTTAEKNGIDIYSLTVDSRVSS...,930.0,626
78000,YYLSCPMESR,10.0,O75326,True,True,"Rydbirk, Duong",1.0,0.714958,MTPPPPGRAAPSAPRARVPGPPARLGLPLRLRLLLLLWAAAASAQG...,666.0,561
78012,YYPASPWVDNSR,12.0,P54289,True,True,"Faigle, Rydbirk, Duong",1.0,0.746092,MAAGCLLALTLTLFQSLLIGPSSEEPFPSAVTIKSWVDKMQEDLVT...,1103.0,216


In [36]:
def filterer(x):
    if len(x['CSF'].value_counts()) == 2:
        if x['CSF'].value_counts()[0] >1 and x['CSF'].value_counts()[1] >1:
            return True
        else: 
            if x['CSF'].value_counts()[0] > 1:
                change_negative.append(x['Protein'].values[0])
            else: 
                change_positive.append(x['Protein'].values[0])
    return False

change_negative = []
change_positive = []
proteingroups = brain_df_filtered.groupby('Protein')
mixed_proteins_filtered = proteingroups.filter(lambda x: filterer(x))
mixed_proteins_filtered.reset_index(inplace=True,drop=True)
print(len(change_negative))
print(len(change_positive))
negatives = pd.DataFrame(change_negative, columns=["Protein"])
negatives.to_csv('austin_db/negative_proteins.csv', index=False)
positives = pd.DataFrame(change_positive, columns=["Protein"])
positives.to_csv('austin_db/positive_proteins.csv', index=False)
print(len(mixed_proteins_filtered['Protein'].unique()))
mixed_proteins_filtered

457
159
1060


,Peptide_Sequence,Sequence_Length,Protein,Brain,CSF,Author,Detectability,Detection_Probability,Protein_Sequence,Protein_Length,Position_in_Protein
0,AAAAAAAAAAAAAAAGAGAGAK,22.0,P55011,True,True,"Faigle, Rydbirk, Duong",0.0,0.122071,MEPRPTAPSSGAPGLAGVGETPSAAALAAARVELPGTAVPSVPEDA...,1212.0,92
1,AAAAAAAAAAAPPAPPEGASPGDSAR,26.0,Q8WXD9,True,False,"Faigle, Rydbirk, Duong",1.0,0.581326,MGKEQELVQAVKAEDVGTAQRLLQRPRPGKAKLLGSTKKINVNFQD...,1431.0,1344
2,AAAAAAALQAK,11.0,P36578,True,True,"Faigle, Rydbirk, Duong",1.0,0.673549,MACARPLISVYSEKGESSGKNVTLPAVFKAPIRPDIVNFVHTNLRK...,427.0,353
3,AAAAAWEEPSSGNGTAR,17.0,Q9P258,True,False,"Faigle, Duong",1.0,0.676112,MPRKKAAAAAWEEPSSGNGTARAGPRKRGGPAGRKRERPERCSSSS...,522.0,5
4,AAAAMAPIK,9.0,P30044,True,False,"Rydbirk, Duong",1.0,0.644743,MGLAGVCALRRSAGYILVGGAGGQSAAAAARRYSEGEWASGGVRSF...,214.0,48
...,...,...,...,...,...,...,...,...,...,...,...
21624,YYGGGYGSTQATFMVFQALAQYQK,24.0,P01024,True,True,"Faigle, Rydbirk",1.0,0.817121,MGPTSGPSLLLLLLTHLPLALGSPMYSIITPNILRLESEETMVLEA...,1663.0,1260
21625,YYIAASYVK,9.0,Q92820,True,True,"Faigle, Rydbirk, Duong",1.0,0.686800,MASPGCLLCVLGLLLCGAASLELSRPHGDTAKKPIIGILMQKCRNK...,318.0,53
21626,YYMNQVEETR,10.0,P52888,True,False,"Rydbirk, Duong",1.0,0.731291,MKPPAACAGDMADAASPCSVVNDLRWDLSAQQIEERTRELIEQTKR...,689.0,338
21627,YYPASPWVDNSR,12.0,P54289,True,True,"Faigle, Rydbirk, Duong",1.0,0.746092,MAAGCLLALTLTLFQSLLIGPSSEEPFPSAVTIKSWVDKMQEDLVT...,1103.0,216


In [37]:
# save peptide sequence df
brain_df_filtered.reset_index(drop=True)
brain_df_filtered.to_csv('austin_db/brain_df_filtered.csv',index=False)  
mixed_proteins_filtered.to_csv('austin_db/mixed_proteins_filtered.csv',index=False)  

# save domain df
proteindomains.to_csv('austin_db/proteindomains.csv',index=False) 
proteinsummaries.to_csv('austin_db/proteinsummaries.csv',index=False)  

In [41]:
print('Before changing classification of mixed overlapping peptides ',brain_df_filtered['CSF'].value_counts()[1],'were true')

# doesnt change the classification of single amino acid theoretical peptides, so not K or R

overlap = overlapsdf.loc[(overlapsdf['TP'] != 'K')]
overlap = overlap.loc[(overlap['TP'] != 'R')]
overlap = overlap[overlap['Classification']=='Mixed']
# iterates over remaining tp's which overlap

for i,row in overlap.iterrows():
    
    # retrieves related tp protein from strict df
    temp = brain_df_filtered[brain_df_filtered['Protein'] == row['Protein']]
    
    # selects peptides which contain the TP
    temp = temp[(temp['Position_in_Protein']<=row['Start']) & (row['Start']<=(temp['Position_in_Protein']+temp['Sequence_Length']))]

    # get index from brain_df_filtered
    for index in temp.index:
        # at location strict in CSF change to positive
        brain_df_filtered.at[index, 'CSF'] = True
print('After changing classification of mixed overlapping peptides ',brain_df_filtered['CSF'].value_counts()[1],'were Brain and CSF')

Before changing classification of mixed overlapping peptides  9953 were true
After changing classification of mixed overlapping peptides  11567 were Brain and CSF


In [42]:
#below here we begin dealing with Theoretical and Region data.

In [43]:
'''
For each protein, checks the classification of its related protein, and classifies the protein as such, (positive if all positive,
negative if all negative, elsewise mixed. Then computes the coverage of theoretical peptides covered by the peptides found)
'''

def TheoreticalCoverage(theogroup):
    # values counts unfound vs found, total and per protein -> coverage in number of peptides found vs theoretical number
    theocoveragedict = {}
    for gr in theogroup:
        protein = gr[1]['Protein'].values[0]
        classificationlist = gr[1]['classificationlist']

        #if true all values in classification list are unfound or positive
        if np.all(np.isin(classificationlist,['Unfound','Positive'])) == True:
            # only positive or unfound
            classi = 'Positive'

        # if true all values in classification list are unfound or Negative
        elif np.all(np.isin(classificationlist,['Unfound','Negative'])) == True:
            # only negative or unfound
            classi = 'Negative'

        else: 
            # mixed
            classi = 'Mixed'

        counterdictio = Counter(classificationlist)
        theocoveragedict.update({protein:[(Counter(classificationlist)['Positive']+Counter(classificationlist)['Negative'])/len(classificationlist)*100,classi,counterdictio['Positive'],counterdictio['Negative'],counterdictio['Unfound']]})

    return theocoveragedict

In [44]:
def transitions(mixed_proteins,theoritical):
    # list of mixed proteins
    mixed = mixed_proteins['Protein'].unique()
    mixeddf = theoritical[theoritical['Protein'].isin(mixed)]
    # loop groups
    transitionlist = []
    for g in mixeddf.groupby('Protein'):
        transitioncounter = 0
        for pep in g[1].iterrows():
            # skips unfound peptides and first peptide
            if   pep[1]['classificationlist'] == 'Unfound' or pd.isnull(pep[1]['Last_Found_Left']):
                continue
            # compares peptide to last found neighbour
            if pep[1]['classificationlist'] != pep[1]['Last_Found_Left']:
                transitioncounter += 1
        transitionlist.append(transitioncounter)
        # check for number of mismatches in neighbouring peptides
    # make note of proteins which only have one negative or positive peptide
    mixed_proteins.loc[:, "Transitions"] = transitionlist
    return mixed_proteins

In [45]:
theogroup = theoriticaldf.groupby('Protein')
theocoverage = pd.DataFrame.from_dict(data=TheoreticalCoverage(theogroup),orient='index',columns=['Coverage','Classification','Positive','Negative','Unfound'])
print(theocoverage['Classification'].value_counts())
theocoverage = theocoverage.rename_axis('Protein').reset_index()
mixed_proteins = theocoverage[theocoverage['Classification']=='Mixed']
transitionsdf = transitions(mixed_proteins,theoriticaldf)

Negative    2896
Mixed       1526
Positive     145
Name: Classification, dtype: int64


/var/folders/zx/9wq85h090yj8pmk6sbvl93n80000gn/T/ipykernel_1500/3719938472.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mixed_proteins.loc[:, "Transitions"] = transitionlist


In [46]:
theoriticaldf
regiondict = {}
index = 0
# create region df
# row for each region in protein, with related protein start, end and whether neighbouring left (N) or right (C) terminal columns and number of matchedpeptides in region, number of theoretical peptides in total in region
for gr in brain_df_filtered.sort_values(['Protein','Position_in_Protein'],ascending=True).groupby('Protein'):
    first = True
    neighbourN = True
    transition = None
    peptidecounter = 0
    for pep in gr[1].iterrows():
        if first == True:
            regionstart = 0
            transition = pep[1]['CSF']
            regionclass = pep[1]['CSF']
            first = False
        if regionclass == pep[1]['CSF']:
            peptidecounter += 1
        if transition != None and transition != pep[1]['CSF']:
            regiondict.update({index:[pep[1]['Protein'],regionclass,regionstart,regionend,neighbourN,False,peptidecounter]})
            index +=1
            peptidecounter = 1
            neighbourN = False
            transition = pep[1]['CSF']
            regionstart = pep[1]['Position_in_Protein']
            regionclass = pep[1]['CSF']
        regionend = pep[1]['Position_in_Protein']+pep[1]['Sequence_Length']-1
        
    regiondict.update({index:[pep[1]['Protein'],regionclass,regionstart,regionend,neighbourN,True,peptidecounter]})
    index +=1
regiondf = pd.DataFrame.from_dict(data=regiondict,orient='index',columns=['Protein','Classification','Start','End','N_Terminal','C_Terminal','Number_Peptides_in_Region'])

In [48]:
def filterclassification(peptides):
    # these peptides are from mixed protein, so if sum = 1, only 1 positive peptide, so classification becomes negative
    if sum(peptides) == 1:
        return 'Negative'
    # if only 1 false peptide, than protein is considered brain + CSF instead of mixed
    elif peptides.value_counts()[False] == 1:
        return 'Positive'
    return'Mixed'
    
    
    
checker = np.isin(brain_df_filtered['Protein'].unique(),csf_df['Protein'])
print(Counter(checker))
brainproteins = brain_df_filtered.sort_values(['Protein','Position_in_Protein'],ascending=True).groupby('Protein',as_index=False)
print(len(brainproteins))
notfound = []
counterdict = {}
negcounter = 0
for group in brainproteins:

    # name of related protein
    protein = group[1]['Protein'].values[0]
    # sequence of related protein
    sequence = group[1]['Protein_Sequence'].values[0]
    # list of related peptides from protein
    peplist = group[1]['Peptide_Sequence']

    overlapcounter = 0
    seqdict = {}

    # creates dictionary with entry for each residue
    for i,l in enumerate(sequence):
        seqdict.update({i:[l,0]}) # initializes count of each amino acid as 0
    
    # loops each peptide from list of related peptides from the protein
    for pep in group[1].iterrows():
        firstaa = True
        for i in range(int(pep[1]['Position_in_Protein']),int(pep[1]['Position_in_Protein'])+int(pep[1]['Sequence_Length'])):
            seqdict[i][1] += 1
            if seqdict[i][1] > 1 and firstaa == True:
                overlapcounter += 1
                firstaa = False

    
    # checks group for any peptides positive or negative class
    # if negative and not positive
    if group[1]['CSF'].all() == True: # if all peptides also found in CSF than classification protein is positive
        #print('postive',group[1]['CSF'].all())
        classification = 'Positive'
        filteredclassification = 'Positive'
    elif sum(group[1]['CSF']) == 0: # if all peptides false then
        classification = 'Negative'
        filteredclassification = 'Negative'
        negcounter +=1
        #if np.isin(protein,csfdf['Protein']):
            #print(protein)
            #print('test:',(~group[1]['CSF']).all())
        #print('negative',~group[1]['CSF'].all())
    else:
        classification = 'Mixed' # if not all true or all false than peptides are mixed
        # determines filter classification
        filteredclassification = filterclassification(group[1]['CSF'])
    
    # counts amino acids not found, thus equal 0
    residuenotfound = sum(x.count(0) for x in seqdict.values())
    # calculates percetange of protein sequence not covered by the peptides
    coverage = ((len(sequence)-residuenotfound)/len(sequence)*100)
    # saves protein accession as key, then coverage, number of peptides, overlaps, average peptide length, protein length, classifications and sequence dict
    counterdict.update({protein:[coverage,sequence,len(group[1]['Peptide_Sequence']),overlapcounter,np.mean(group[1]['Sequence_Length']),len(sequence),classification,filteredclassification,seqdict]})
print('notfound',notfound)
print('number of negative protiens:',negcounter)

Counter({False: 3194, True: 2176})
5370
notfound []
number of negative protiens: 3499


In [49]:
print(regiondf['Number_Peptides_in_Region'].value_counts())
filteredregion = regiondf[regiondf['Number_Peptides_in_Region']!=1]
filteredregion.reset_index(inplace=True)
filteredregion

1      3290
2      1931
3      1243
4       836
5       568
       ... 
88        1
83        1
79        1
61        1
100       1
Name: Number_Peptides_in_Region, Length: 73, dtype: int64


,index,Protein,Classification,Start,End,N_Terminal,C_Terminal,Number_Peptides_in_Region
0,0,A0A096LPE2,True,0,142.0,True,True,2
1,1,A0A0B4J2A0,False,0,425.0,True,True,8
2,2,A0A0C4DH38,False,0,116.0,True,True,3
3,3,A0AVT1,False,0,1043.0,True,True,48
4,4,A0FGR8,False,0,718.0,True,False,16
...,...,...,...,...,...,...,...,...
8237,11526,Q9Y6W5,False,0,444.0,True,True,8
8238,11527,Q9Y6X4,False,0,649.0,True,True,20
8239,11528,Q9Y6X5,False,0,53.0,True,False,2
8240,11530,Q9Y6X5,False,69,309.0,False,True,4


In [50]:
proteinregs = filteredregion.groupby('Protein')
regionclassification = {}
for p in regiondf['Protein'].unique():
    if p in filteredregion['Protein'].unique():
        protein = proteinregs.get_group(p)
        values = protein['Classification'].values
        values[values == 'Negative'] = False
        values[values == 'Positive'] = True
        if (values).all() == True:
            classification = 'Positive'
        elif sum(values) == 0:
            classification = 'Negative'
        elif sum(values) != 0:
            classification = 'Mixed'
        else:
            classification = 'None'
    else:
        classification = 'None'
    regionclassification[p] = classification
regionclassification

/var/folders/zx/9wq85h090yj8pmk6sbvl93n80000gn/T/ipykernel_1500/2121330168.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  values[values == 'Negative'] = False
/var/folders/zx/9wq85h090yj8pmk6sbvl93n80000gn/T/ipykernel_1500/2121330168.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  values[values == 'Positive'] = True


{'A0A096LPE2': 'Positive',
 'A0A0B4J2A0': 'Negative',
 'A0A0C4DH38': 'Negative',
 'A0AVT1': 'Negative',
 'A0FGR8': 'Negative',
 'A0JNW5': 'Negative',
 'A0MZ66': 'Mixed',
 'A1A5C7': 'Negative',
 'A1IGU5': 'Negative',
 'A1L0T0': 'Negative',
 'A1X283': 'Negative',
 'A1Z1Q3': 'Negative',
 'A2RTX5': 'Negative',
 'A2RU30': 'Negative',
 'A2RU67': 'Negative',
 'A3KMH1': 'Negative',
 'A4D126': 'Negative',
 'A4D161': 'Negative',
 'A4D1E9': 'Negative',
 'A4D1P6': 'Negative',
 'A4D2B0': 'Negative',
 'A5D8V6': 'Negative',
 'A5PKW4': 'Negative',
 'A5PLN9': 'Negative',
 'A5YM72': 'Negative',
 'A6NCS6': 'Negative',
 'A6NDB9': 'Negative',
 'A6NDG6': 'Negative',
 'A6NDU8': 'Negative',
 'A6NE02': 'Negative',
 'A6NGB9': 'Negative',
 'A6NGN9': 'Positive',
 'A6NHR9': 'Negative',
 'A6NHX0': 'Negative',
 'A6NIH7': 'Negative',
 'A6NKN8': 'None',
 'A6NL88': 'Negative',
 'A6ZKI3': 'Negative',
 'A7KAX9': 'Negative',
 'A8MU93': 'Negative',
 'A8MVW0': 'Negative',
 'A8MWD9': 'Negative',
 'A8MXV4': 'Negative',
 'B4DS

In [51]:
coveragedf = pd.DataFrame.from_dict(data=counterdict,orient='index',columns=['Coverage','Protein_Sequence','Number_Peptides','Overlapping_Peptides','Mean_Peptide_Length','Protein_Length','Classification','Filtered_Classification','Seqdict'])

coveragedf = coveragedf.rename_axis('Protein').reset_index()

coveragedf['Region_Classification'] = coveragedf['Protein'].apply(lambda i: regionclassification[i])


brain_df_filtered["Protein_Classification"] = brain_df_filtered['Protein'].apply(lambda x: coveragedf[coveragedf['Protein']==x]['Classification'].values[0])
brain_df_filtered["Region_Classification"] = brain_df_filtered['Protein'].apply(lambda y: coveragedf[coveragedf['Protein']==y]['Region_Classification'].values[0])
brain_df_filtered["Filtered_Classification"] = brain_df_filtered['Protein'].apply(lambda y: coveragedf[coveragedf['Protein']==y]['Filtered_Classification'].values[0])
brain_df_filtered

/var/folders/zx/9wq85h090yj8pmk6sbvl93n80000gn/T/ipykernel_1500/3554925065.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brain_df_filtered["Protein_Classification"] = brain_df_filtered['Protein'].apply(lambda x: coveragedf[coveragedf['Protein']==x]['Classification'].values[0])
/var/folders/zx/9wq85h090yj8pmk6sbvl93n80000gn/T/ipykernel_1500/3554925065.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brain_df_filtered["Region_Classification"] = brain_df_filtered['Protein'].apply(lambda y: coveragedf[

,Peptide_Sequence,Sequence_Length,Protein,Brain,CSF,Author,Detectability,Detection_Probability,Protein_Sequence,Protein_Length,Position_in_Protein,Protein_Classification,Region_Classification,Filtered_Classification
0,AAAAAAAAAAAAAAAGAGAGAK,22.0,P55011,True,True,"Faigle, Rydbirk, Duong",0.0,0.122071,MEPRPTAPSSGAPGLAGVGETPSAAALAAARVELPGTAVPSVPEDA...,1212.0,92,Mixed,Mixed,Mixed
1,AAAAAAAAAAAPPAPPEGASPGDSAR,26.0,Q8WXD9,True,False,"Faigle, Rydbirk, Duong",1.0,0.581326,MGKEQELVQAVKAEDVGTAQRLLQRPRPGKAKLLGSTKKINVNFQD...,1431.0,1344,Mixed,Negative,Mixed
2,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29.0,P37108,True,False,"Rydbirk, Duong",1.0,0.807786,MVLLESEQFLTELTRLFQKCRTSGSVYITLKKYDGRTKPIPKKGTV...,136.0,107,Mixed,Mixed,Mixed
3,AAAAAAAAVPSAGPAGPAPTSAAGR,25.0,Q9Y4H2,True,True,"Faigle, Duong",1.0,0.501463,MASPPRHGPPGPASGDGPNLNNNNNNNNHSVRKCGYLRKQKHGHKR...,1338.0,693,Mixed,Negative,Negative
4,AAAAAAALQAK,11.0,P36578,True,True,"Faigle, Rydbirk, Duong",1.0,0.673549,MACARPLISVYSEKGESSGKNVTLPAVFKAPIRPDIVNFVHTNLRK...,427.0,353,Mixed,Mixed,Mixed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78020,YYQTIGNHASYYK,13.0,Q9UNM6,True,False,"Faigle, Rydbirk",1.0,0.684231,MKDVPGFLQQSQNSGPGQPAVWHRLEELYTKKLWHQLTLQVLDFVQ...,376.0,161,Negative,Negative,Negative
78021,YYRPTEVDFLQGDCTK,16.0,O60547,True,False,"Rydbirk, Duong",1.0,0.785365,MAHAPARCPSARGSGDGEMGKPRNVALITGITGQDGSYLAEFLLEK...,372.0,322,Negative,Negative,Negative
78022,YYSDIGK,7.0,Q9HCM2,True,False,"Faigle, Rydbirk, Duong",1.0,0.780594,MKAMPWNWTCLLSHLLMVGMGSSTLLTRQPAPLSQKQRSFVTFRGE...,1894.0,1813,Mixed,Negative,Mixed
78023,YYSDLFSYCDIESTK,15.0,Q96D71,True,False,"Rydbirk, Duong",1.0,0.740213,MEGLTLSDAEQKYYSDLFSYCDIESTKKVVVNGRVLELFRAAQLPN...,796.0,12,Mixed,Negative,Negative


In [53]:
brain_df_filtered.loc[brain_df_filtered['CSF'] == True, 'CSF'] = 'Brain + CSF'
brain_df_filtered.loc[brain_df_filtered['CSF'] == False, 'CSF'] = 'Brain Only'
brain_df_filtered

/var/folders/zx/9wq85h090yj8pmk6sbvl93n80000gn/T/ipykernel_1500/758614403.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brain_df_filtered.loc[brain_df_filtered['CSF'] == True, 'CSF'] = 'Brain + CSF'


,Peptide_Sequence,Sequence_Length,Protein,Brain,CSF,Author,Detectability,Detection_Probability,Protein_Sequence,Protein_Length,Position_in_Protein,Protein_Classification,Region_Classification,Filtered_Classification
0,AAAAAAAAAAAAAAAGAGAGAK,22.0,P55011,True,Brain + CSF,"Faigle, Rydbirk, Duong",0.0,0.122071,MEPRPTAPSSGAPGLAGVGETPSAAALAAARVELPGTAVPSVPEDA...,1212.0,92,Mixed,Mixed,Mixed
1,AAAAAAAAAAAPPAPPEGASPGDSAR,26.0,Q8WXD9,True,Brain Only,"Faigle, Rydbirk, Duong",1.0,0.581326,MGKEQELVQAVKAEDVGTAQRLLQRPRPGKAKLLGSTKKINVNFQD...,1431.0,1344,Mixed,Negative,Mixed
2,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29.0,P37108,True,Brain Only,"Rydbirk, Duong",1.0,0.807786,MVLLESEQFLTELTRLFQKCRTSGSVYITLKKYDGRTKPIPKKGTV...,136.0,107,Mixed,Mixed,Mixed
3,AAAAAAAAVPSAGPAGPAPTSAAGR,25.0,Q9Y4H2,True,Brain + CSF,"Faigle, Duong",1.0,0.501463,MASPPRHGPPGPASGDGPNLNNNNNNNNHSVRKCGYLRKQKHGHKR...,1338.0,693,Mixed,Negative,Negative
4,AAAAAAALQAK,11.0,P36578,True,Brain + CSF,"Faigle, Rydbirk, Duong",1.0,0.673549,MACARPLISVYSEKGESSGKNVTLPAVFKAPIRPDIVNFVHTNLRK...,427.0,353,Mixed,Mixed,Mixed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78020,YYQTIGNHASYYK,13.0,Q9UNM6,True,Brain Only,"Faigle, Rydbirk",1.0,0.684231,MKDVPGFLQQSQNSGPGQPAVWHRLEELYTKKLWHQLTLQVLDFVQ...,376.0,161,Negative,Negative,Negative
78021,YYRPTEVDFLQGDCTK,16.0,O60547,True,Brain Only,"Rydbirk, Duong",1.0,0.785365,MAHAPARCPSARGSGDGEMGKPRNVALITGITGQDGSYLAEFLLEK...,372.0,322,Negative,Negative,Negative
78022,YYSDIGK,7.0,Q9HCM2,True,Brain Only,"Faigle, Rydbirk, Duong",1.0,0.780594,MKAMPWNWTCLLSHLLMVGMGSSTLLTRQPAPLSQKQRSFVTFRGE...,1894.0,1813,Mixed,Negative,Mixed
78023,YYSDLFSYCDIESTK,15.0,Q96D71,True,Brain Only,"Rydbirk, Duong",1.0,0.740213,MEGLTLSDAEQKYYSDLFSYCDIESTKKVVVNGRVLELFRAAQLPN...,796.0,12,Mixed,Negative,Negative


In [56]:
regiondf.to_csv('austin_db/regiondf.csv',index=False)  
filteredregion.to_csv('austin_db/filtered_regiondf.csv',index=False)  
brain_df_filtered.to_csv('austin_db/final_brain_df.csv',index=False)

# save coverage df
coveragedf.to_csv('austin_db/coverage.csv',index=False)

# save theocoverage
theocoverage.to_csv('austin_db/theocoverage.csv',index=False)

# df with all theoritical sequences
theoriticaldf.to_csv('austin_db/theoriticaldf.csv',index=False)

# df with all theoritical sequences
transitionsdf.to_csv('austin_db/transitionsdf.csv',index=False)  